# Word Generation In $G=\mathbb{Z}_d^{*m},\ d>2$

### Define, for all $i\in\mathbb{Z}_{>0}$, $x_i$, with the relation $x_i^d=1$. Write $G=*_{i=1}^m\langle x_i|x_i^d=1\rangle$, with generating set $S=\bigcup_{i=1}^m S_i$, with $S_i=\{x_i\}$.

Our interest is to generate all the **excursions** on the Cayley Graph of $G$ wrt $S$. These are words that end up back at one (Eg. $x_1x_2x_3x_3x_3x_2x_2x_1^{-1}$ for $d=3$ ). To avoid redundancy, we can generate **simple excursions**, those that don't go back to 1 "too early".

Let $Z_{g,X}$ be the set of words ($\in S^*$) that evaluates to $g\in G$ and contains no proper prefix in $X\subseteq G$.
Let $Z_{g,X}^{(n)}$ be such words of length $n$. In most cases, we only need $X\subseteq S\cup\{1\}$.

Our Goal is to find $Z_{1,\emptyset}^{(n)}$ for some small $n$. Contenations of such words of varied $n$ gives another such word. To avoid redundency, We could compute $Z_{1,\{1\}}^{(n)}$, and notice that 

$$Z_{1,\emptyset}^{(n)}=\left(\cup_{\lambda}[\times_{i}Z_{1,\{1\}}^{(\lambda_i)}]\right)\cup Z_{1,\{1\}}^{(n)}$$

where $\lambda=(\lambda_i)$ ranges over the $2^{n-1}-1$ nontrivial compositions of $n$. Furthermore, the $2^{n-1}$ sets involved in the union expression are pairwise disjoint. 

In fact, there is an even simpler way. For each $1\le r \le n$, Compute, ___assuming___ $m=r$, the elements $w$ of $Z_{1,\{1\}}^{(n)}$ such that, for each $i=1,2,...,r$, we can write $w=ux_iv$, where $u\in \{x_1,..., x_{i-1}\}^*$. Equivalently, for $1\le i<j \le r$, the first occurence of $x_i$ is to the left of the first occurence of $x_j$. Call these **canonical simple excursions**. Let $\bar{P}_{n,r}$ denote the number of such elements. Let $\bar{P}_{n}(m)=|Z_{1,\{1\}}^{(n)}|$, and $P_{n}(m)=|Z_{1,\emptyset}^{(n)}|$ for any given $m$. We are interested in obtaining $P_n(m)$.

We have the amazing identity  $$\bar{P}_n(m)=\sum_{r=1}^n \bar{P}_{n,r}\cdot (m)_r,$$ where $(t)_r=t(t-1)(t-2)...(t-r+1)\in \mathbb{Z}[t]$ is the falling factorial polynomial of degree $r$. [Jonah: Prove this identity]


Notice that $\bar{P}_{n,r}>0$ if $d|n$ and either $r=n=0$ or $1\le r\le \frac{n}{d}$,<br>
(Why is this? **Hint:** What do simple excursions that start and end with $x_1$ look like in general?) 
and $\bar{P}_{n,r}=0$ otherwise.

Thus, $\bar{P}_{n}(m)$ is 0 for odd $n$, and a degree $\frac{n}{d}$ polynomial in $m$ for even $n$. Same can be said for $P_n(m)$ (Explain why).


Here, we use the integer $i$, to denote $x_i$. Words will be represented as tuples of integers.

In [65]:
var('m')
d=4

In [66]:
def char_inv(c):
    '''
    Returns the word that maps to the inverse element of c
    This is c^(d-1) in a tuple representation
    '''
    return (c,)*(d-1)
    


class Word_Zdm(object):
    def __init__(self,w=[]):
        self.w = []
        self.wred = [] # reduced word of w
        for c in w:
            self.add_char(c)
    
    def add_char(self,c):
        w = self.w
        wred = self.wred
        w.append(c)
        for i in (1..d-1):
            if i>len(wred) or wred[-i]!=c:
                wred.append(c)
                break
        else:
            wred[-d+1:]=[]
    
    def del_last(self):
        # assume w is nonempty
        w = self.w
        wred = self.wred
        c = w.pop()
        if wred[-1:]==[c]:
            wred.pop()
        else:
            wred.extend(char_inv(c))
        return c
        
    def last(self):
        return self.w[-1]
        
    def word(self):
        return tuple(self.w)
    
    def reduced_word(self):
        return tuple(self.wred)
    
    def __len__(self):
        return len(self.w)
    
    def reduced_length(self):
        return len(self.wred)
        
    def __str__(self):
        return str(self.word())
    
    def __iter__(self):
        return iter(self.w)
    
    def clone(self):
        # bit more efficient than calling Word_Z2Z(self)
        other = Word_Z2Z()
        other.w = list(self.w)
        other.wred = list(self.wred)
        return other
    
    def clear(self):
        self.w = []
        self.wred = []

In [67]:
wgen = Word_Zdm()

In [68]:
from collections import defaultdict
import time
from datetime import timedelta

In [69]:
KEY = 16
def hash_word(w):
    h = 0
    for c in w:
        h *= KEY
        h += c
    return h

def unhash_word(h):
    w = []
    while h:
        w.insert(0,h%KEY)
        h//=KEY
    return tuple(w)



In [70]:
def gen_canonical_simple_excursions_up_to_length(n, wgen, wlist, maxindex=1, cont_mi=False):
    '''
        Populates wlist, assumed to be defaultdict(list) object with (k,r) missing key for each 0<=k<=n, 0<=r<=n/d.
        Upon termination, wlist[(k,r)] is a sorted list canonical simple excursions of G when m=r.
        Parameters:
            n: maximum length of all words to be considered
            wgen: A Word_Zdm instance, assumed empty
            maxindex: maximum possible i, so that x_i can be appended to the current word
            wlist: defaultdict(list) object
            cont_mi: bool type, True iff currect word contains the maximum index allowed
        Return:
            None
    '''
    if not wgen.reduced_length():
        wlist[(ZZ(len(wgen)), ZZ(maxindex-int(not cont_mi)))].append((wgen.word()))
        if len(wgen):
            return
    if len(wgen)<n and wgen.reduced_length()+len(wgen)*(d-1)<=n*(d-1):
        for i in range(1,maxindex+1):
            # standard backtracking approach
            wgen.add_char(i)
            # assume i=maxindex, so that nxt_mi=i+1 is not yet visited
            flag = False
            nxt_mi = i+1
            if nxt_mi<=maxindex:
                # assumption wrong. i is still too small. 
                # If maxindex has not appeared up to now, it will not appear with the additional character.
                nxt_mi=maxindex
                flag = cont_mi
            if nxt_mi>n//d:
                # Gone too far, cannot come back in time
                # Can cut the next index by the pigeonhole principle.
                nxt_mi = maxindex
                flag = True
            gen_canonical_simple_excursions_up_to_length(n,wgen,wlist,nxt_mi, flag)
            wgen.del_last()
        
        

In [82]:
maxlen =(15//d)*d
cse_by_len = defaultdict(list)
gen_canonical_simple_excursions_up_to_length(maxlen,wgen,cse_by_len)
wgen.clear()

In [83]:
for (n,r) in sorted(cse_by_len):
    print(n,r,len(cse_by_len[(n,r)]))
print(cse_by_len)
print(sum(map(len, cse_by_len.values())))

0 0 1
4 1 1
8 2 3
12 2 15
12 3 15
defaultdict(<class 'list'>, {(0, 0): [()], (4, 1): [(1, 1, 1, 1)], (12, 2): [(1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1), (1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1), (1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1), (1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1), (1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1), (1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1), (1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1), (1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1), (1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1), (1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1), (1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1), (1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1), (1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1), (1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1), (1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1)], (8, 2): [(1, 1, 1, 2, 2, 2, 2, 1), (1, 1, 2, 2, 2, 2, 1, 1), (1, 2, 2, 2, 2, 1, 1, 1)], (12, 3): [(1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 1), (1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 2, 1), (1, 1, 1, 2, 2, 3, 3, 3, 3, 2, 2, 1), (1, 1, 1, 2, 3, 3, 3, 3, 2, 2, 2, 1), (1, 1, 2, 2, 2, 2, 1, 3, 3, 3, 3, 1), (1, 1, 2, 2, 2, 2, 3, 3, 3, 3,

In [84]:


def time_test(func, params):
    runtimes = []
    for p in params:
        start = time.time()
        func(p)
        end = time.time()
        runtimes.append((p, end-start))
    return runtimes
    

In [85]:
wgen.clear()
runtimes = time_test(lambda n: gen_canonical_simple_excursions_up_to_length(n,wgen,defaultdict(list)), range(d,5*d+1,d))
runtimes

KeyboardInterrupt: 

In [55]:
for n, dur in runtimes:
    print('%2d\t%7s'%(n, timedelta(seconds=dur)))

 3	0:00:00.000478
 6	0:00:00.001374
 9	0:00:00.031894
12	0:00:01.048270
15	0:00:51.016460


In [86]:
# save and load routines

def _preconvert(save_mode, hashfn):
    def _preconvert_word(w):
        w = hashfn(w)
        if save_mode:
            return 'e' if not str(w) else str(w)
        return '' if w=='e' else w
    return _preconvert_word

def save_gen(fname, wlist, hashfn=lambda x: x, maxlen=None):
    if maxlen is None:
        maxlen = max([n for n,r in wlist if wlist[(n,r)]])
    with open(fname,'w') as handle:
        for n in range(maxlen+1):
            for r in range(n//2+1):
                if (n,r) in wlist:
                    handle.write('%d %d %s\n'%(n,r,' '.join(map(_preconvert(1,hashfn),wlist[(n,r)]))) )

def load_gen(fname, unhashfn = lambda x: x, skip_empty = 0,maxlen = None):
    wlist = {}
    with open(fname) as handle:
        for line in handle:
            line = line.strip()
            if line:
                wn = line.split()
                n = ZZ(wn.pop(0))
                r = ZZ(wn.pop(0))
                if skip_empty and not wn:
                    continue
                if maxlen is not None and n>maxlen:
                    continue
                wlist[(n,r)] = list(map(_preconvert(0,unhashfn), map(ZZ,wn)))
    return defaultdict(list, wlist)


In [89]:
save_gen('cseZ%dstar_m.txt'%d, cse_by_len, hash_word)

In [90]:
# loaded_cse_by_len = load_gen('cseZ2star_m.txt', unhash_word, 1)
# for (n,r) in sorted(cse_by_len):
#     print(n,r,len(loaded_cse_by_len[(n,r)]))
#     if  loaded_cse_by_len[(n,r)] != cse_by_len[(n,r)]:
#         print('\t', loaded_cse_by_len[(n,r)][:5], cse_by_len[(n,r)][:5])
# print(set(cse_by_len)==set(loaded_cse_by_len))
# print({(type(n),type(r)) for n,r in cse_by_len})

cse_by_len = load_gen('cseZ%dstar_m.txt'%d, unhash_word, 1)


Now we compute $P_n(m)$ for $n=0,1,...,maxlen$. Obesrve that 
$$P_n(m)=\sum_{\sum{\lambda_i}=n,\ \lambda_i>0}\prod_{i}\bar{P}_{\lambda_i}^*(m)=\sum_{\sum{\mu_i}=\frac{n}{d},\ \mu_i>0}\prod_{i}\bar{P}_{d\mu_i}^*(m)$$ for $n$ a multiple of $d$. Of course, we first need to compute $\bar{P}_n(m)$.

In [91]:
def print_counts(ctlist,title=''):
    if title:
        pretty_print(LatexExpr('\\textbf{%s}'%title))
    for n, ct in enumerate(ctlist):
        pretty_print(LatexExpr('n=%d:'%n),ct)


In [93]:
se_counts_by_length = [1,0]

while len(se_counts_by_length)<=maxlen:
    ct = 0
    n = len(se_counts_by_length)
    n = ZZ(n)
    if n%d:
        se_counts_by_length.append(ct)
        continue
    for r in (0..n/d):
        if (n,r) in cse_by_len:
            ct += ZZ(len(cse_by_len[(n,r)]))*falling_factorial(m,r)
    ct = ct.expand().simplify_full()
    se_counts_by_length.append(ct)

In [94]:
print_counts(se_counts_by_length, 'Simple Excursion Count')

\textbf{Simple Excursion Count}

n=0: 1

n=1: 0

n=2: 0

n=3: 0

...

n=8: 3*m^2 - 3*m

n=9: 0

n=10: 0

n=11: 0

n=12: 15*m^3 - 30*m^2 + 15*m

In [95]:
exc_counts_by_length = [1,0]
while len(exc_counts_by_length)<=maxlen:
    ct = 0
    n = len(exc_counts_by_length)
    n = ZZ(n)
    if n%d:
        exc_counts_by_length.append(ct)
        continue
    
    for miu in Compositions(ZZ(n/d)):
        term = 1
        for el in miu:
            term *= se_counts_by_length[el*d]
        ct+= term
    ct= ct.expand()
    exc_counts_by_length.append(ct)

In [96]:
print_counts(exc_counts_by_length, 'Excursion Count')

\textbf{Excursion Count}

n=0: 1

n=1: 0

n=2: 0

n=3: 0

...

n=8: 4*m^2 - 3*m

n=9: 0

n=10: 0

n=11: 0

n=12: 22*m^3 - 36*m^2 + 15*m